In [12]:
%config InlineBackend.figure_format = 'retina'
%reload_ext autoreload
%autoreload 2
%matplotlib inline
import pandas as pd
from fastai.vision import *
from pathlib import Path
torch.cuda.set_device(1)

In [20]:
image_path_orig = Path('/data/kaggle/met/images')
image_path_tmp = Path('/data/kaggle/met/tmp/images')
image_path_sm = image_path_tmp / '64'
image_path_md = image_path_tmp / '128'
image_path_lg = image_path_tmp / '256'
image_path_xl = image_path_tmp / '512'

In [5]:
met_image_data = '/data/kaggle/met/photographs-02-08-19.csv'
met_df = pd.read_csv(met_image_data)
def extract_image_path(row):
    object_id = row['object_id']
    return f'{object_id}.jpg'
met_df['image_path'] = met_df.apply (lambda row: extract_image_path(row),axis=1)

In [15]:
photo_tfms = get_transforms(flip_vert=True, max_lighting=0.1, max_zoom=1.1, max_warp=0.2, max_rotate=20.0)
fastai_image_items = ImageItemList.from_folder(Path('/data/kaggle/met/images/'))

In [21]:
def resize_one(fn, i, path, size):
    dest = path/fn.relative_to(image_path_orig)
    dest.parent.mkdir(parents=True, exist_ok=True)
    img = PIL.Image.open(fn)
    targ_sz = resize_to(img, size, use_min=True)
    img = img.resize(targ_sz, resample=PIL.Image.BICUBIC).convert('RGB')
    img.save(dest, quality=80)

In [22]:
sets = [(image_path_sm, 64), (image_path_md, 128), (image_path_lg, 256)]
for p,size in sets:
    if not p.exists(): 
        print(f"resizing to {size} into {p}")
        parallel(partial(resize_one, path=p, size=size), fastai_image_items.items)

resizing to 64 into /data/kaggle/met/tmp/images/64


resizing to 128 into /data/kaggle/met/tmp/images/128


resizing to 256 into /data/kaggle/met/tmp/images/256


In [17]:
fastai_image_items.items

array([PosixPath('/data/kaggle/met/images/412237.jpg'), PosixPath('/data/kaggle/met/images/283065.jpg'),
       PosixPath('/data/kaggle/met/images/644306.jpg'), PosixPath('/data/kaggle/met/images/14988.jpg'), ...,
       PosixPath('/data/kaggle/met/images/263237.jpg'), PosixPath('/data/kaggle/met/images/261827.jpg'),
       PosixPath('/data/kaggle/met/images/268170.jpg'), PosixPath('/data/kaggle/met/images/644454.jpg')], dtype=object)